<a href="https://colab.research.google.com/github/gershomrichardbruno/Ergonomics-Smart-Chair/blob/main/Ergonomics_LSTM_Posture_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🪑 Ergonomics Chair: Posture Classification with LSTM
This updated notebook adds LSTM-based classification to detect posture types like upright, slouched, and leaning from joint positions.

In [1]:
!pip install mediapipe opencv-python matplotlib pandas scikit-learn tensorflow

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape
from google.colab import files
from google.colab.patches import cv2_imshow

## 📤 Upload Subject Video

In [3]:
uploaded = files.upload()

Saving 4937030-uhd_2160_3840_24fps.mp4 to 4937030-uhd_2160_3840_24fps.mp4


In [4]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
video_path = list(uploaded.keys())[0]
cap = cv2.VideoCapture(video_path)
landmarks_all = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks:
        joints = []
        for lm in results.pose_landmarks.landmark:
            joints.extend([lm.x, lm.y, lm.z])
        landmarks_all.append(joints)
cap.release()
landmarks_array = np.array(landmarks_all)

## 🏷️ Manually Label Postures (0: Upright, 1: Leaned, 2: Slouched)

In [5]:
# Example: Assign fake labels for demo purposes
num_samples = len(landmarks_array)
y_labels = np.zeros(num_samples)
y_labels[int(num_samples/3):int(2*num_samples/3)] = 1
y_labels[int(2*num_samples/3):] = 2

## 🤖 Train LSTM for Posture Classification

In [6]:
X = landmarks_array
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_seq = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])
y = pd.get_dummies(y_labels).values
X_train, X_test, y_train, y_test = train_test_split(X_seq, y, test_size=0.2)

model = Sequential([
    LSTM(64, input_shape=(1, X_scaled.shape[1])),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - accuracy: 0.5188 - loss: 1.0771 - val_accuracy: 0.7317 - val_loss: 0.9986
Epoch 2/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7264 - loss: 0.9703 - val_accuracy: 0.8049 - val_loss: 0.9014
Epoch 3/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8702 - loss: 0.8671 - val_accuracy: 0.8293 - val_loss: 0.8038
Epoch 4/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8913 - loss: 0.7586 - val_accuracy: 0.8293 - val_loss: 0.6932
Epoch 5/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9110 - loss: 0.6370 - val_accuracy: 0.8049 - val_loss: 0.5950
Epoch 6/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8815 - loss: 0.5452 - val_accuracy: 0.8537 - val_loss: 0.5091
Epoch 7/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8819 - loss: 0.4738 - val_accuracy: 0.8293 - val_loss: 0.4207
Epoch 8/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9219 - loss: 0.3703 - val_accuracy: 0.8537 - val_loss: 0.3601
Epoch 9/15